In [1]:
# Cell 1 — Imports & paths

from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

# If gensim isn't installed yet: in a separate cell run:
# %pip install gensim

from gensim.models import Word2Vec

# Make plots a bit nicer
plt.style.use("ggplot")

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

INTERACTIONS_PATH = DATA_PROCESSED / "interactions.parquet"
TRACK_META_PATH = DATA_PROCESSED / "track_metadata.csv"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("INTERACTIONS_PATH:", INTERACTIONS_PATH)
print("TRACK_META_PATH:", TRACK_META_PATH)


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Cell 2 — Load interactions & metadata

inter = pd.read_parquet(INTERACTIONS_PATH)
track_meta = pd.read_csv(TRACK_META_PATH)

print("Raw interactions shape:", inter.shape)
display(inter.head())

print("\ntrack_metadata.csv shape:", track_meta.shape)
display(track_meta.head())


In [ ]:
# Cell 3 — Basic sanity checks

n_playlists = inter["pid"].nunique()
n_tracks = inter["track_id"].nunique()

print(f"Num playlists in interactions: {n_playlists:,}")
print(f"Num unique tracks in interactions: {n_tracks:,}")

# Sort interactions inside playlists by pos (or duration if you prefer)
inter = inter.sort_values(["pid", "pos"]).reset_index(drop=True)

# Meta sanity
print("\nMetadata coverage of interaction tracks:")
covered = inter["track_id"].isin(track_meta["track_id"]).mean() * 100
print(f"  % of interaction track_ids that have metadata: {covered:.2f}%")


In [ ]:
# Cell 4 — Build playlist sentences (track sequences)

MIN_LEN = 5  # drop very tiny playlists for training stability

playlist_groups = inter.groupby("pid")["track_id"].apply(list)

print("Total playlists:", len(playlist_groups))

playlist_sentences = [tracks for tracks in playlist_groups if len(tracks) >= MIN_LEN]

print(f"Playlists kept (len >= {MIN_LEN}): {len(playlist_sentences)}")
print("Example playlist sentence (first 15 tracks):")
print(playlist_sentences[0][:15])


In [ ]:
# Cell 5 — Train Item2Vec model

# Hyperparameters (tune later if you want)
EMBED_DIM = 64        # size of embedding vector
WINDOW = 20           # how many neighbours left/right to consider
MIN_COUNT = 2         # ignore tracks that appear < 2 times
SG = 1                # 1 = skip-gram, 0 = CBOW
NEGATIVE = 10         # negative samples
EPOCHS = 10

print("Training Word2Vec (Item2Vec) model...")
print(f"  sentences: {len(playlist_sentences):,}")
print(f"  embedding dim: {EMBED_DIM}, window: {WINDOW}, min_count: {MIN_COUNT}")

model = Word2Vec(
    sentences=playlist_sentences,
    vector_size=EMBED_DIM,
    window=WINDOW,
    min_count=MIN_COUNT,
    workers=4,
    sg=SG,
    negative=NEGATIVE,
    epochs=EPOCHS,
)

print("Done!")

# Save model so you can reload later without retraining
MODELS_DIR = PROJECT_ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True)

ITEM2VEC_PATH = MODELS_DIR / "item2vec_word2vec.model"
model.save(str(ITEM2VEC_PATH))
print("Saved model to:", ITEM2VEC_PATH)


In [ ]:
# Cell 6 — Inspect vocabulary & coverage

vocab_tracks = set(model.wv.index_to_key)
print(f"Tracks in Item2Vec vocab (after min_count): {len(vocab_tracks):,}")

inter_tracks = set(inter["track_id"].unique())
meta_tracks = set(track_meta["track_id"].unique())

print(f"Tracks in interactions: {len(inter_tracks):,}")
print(f"Tracks in track_metadata: {len(meta_tracks):,}")

print("\nCoverage:")
print(f"% of interaction tracks in vocab: {len(inter_tracks & vocab_tracks) / len(inter_tracks) * 100:.2f}%")
print(f"% of vocab tracks with metadata: {len(vocab_tracks & meta_tracks) / len(vocab_tracks) * 100:.2f}%")


In [ ]:
# Cell 7 — Helper: find track IDs by name substring

# Keep a slim view of metadata for display
meta_simple = (
    track_meta[["track_id", "track_name", "artist_name", "album_name"]]
    .drop_duplicates()
    .set_index("track_id")
)

def find_track_ids_by_name(query, top_n=20):
    """
    Simple case-insensitive substring search over track_name and artist_name.
    Returns a small DataFrame of candidate tracks.
    """
    q = query.lower()
    mask = (
        track_meta["track_name"].str.lower().str.contains(q, na=False)
        | track_meta["artist_name"].str.lower().str.contains(q, na=False)
    )
    candidates = (
        track_meta.loc[mask, ["track_id", "track_name", "artist_name", "album_name"]]
        .drop_duplicates()
        .head(top_n)
    )
    return candidates

# Example:
display(find_track_ids_by_name("ocean eyes"))


In [ ]:
# Cell 8 — Recommend similar songs via Item2Vec

def describe_tracks(track_ids):
    """
    Given a list/array of track_ids, return a table with names.
    """
    ids = list(track_ids)
    df = meta_simple.loc[meta_simple.index.intersection(ids)].copy()
    df = df.reset_index()
    # Keep the same order as ids
    df = df.set_index("track_id").loc[ids].reset_index()
    return df

def recommend_by_name_item2vec(query, candidate_index=0, top_k=15):
    """
    1) Search for tracks whose name/artist match `query`.
    2) Let you pick one by index (`candidate_index` from the search table).
    3) Use Item2Vec to find the most similar tracks.
    4) Return a DataFrame of recommended tracks + similarity.
    """
    candidates = find_track_ids_by_name(query, top_n=50)
    if candidates.empty:
        print(f"No matches found for query '{query}'")
        return pd.DataFrame()
    
    print("Search results:")
    display(candidates.reset_index(drop=True))

    if candidate_index >= len(candidates):
        raise ValueError(f"candidate_index {candidate_index} out of range (num results={len(candidates)})")
    
    seed_row = candidates.iloc[candidate_index]
    seed_tid = seed_row["track_id"]
    
    print("\nChosen seed track:")
    display(seed_row.to_frame().T)

    if seed_tid not in model.wv:
        print("\n[Item2Vec] Seed track is not in vocab (too rare for MIN_COUNT). "
              "Try another candidate or lower MIN_COUNT and retrain.")
        return pd.DataFrame()

    # Get most similar tracks in embedding space
    sims = model.wv.most_similar(seed_tid, topn=top_k)
    rec_ids = [tid for tid, score in sims]
    rec_scores = [score for tid, score in sims]

    rec_df = describe_tracks(rec_ids)
    rec_df["item2vec_similarity"] = rec_scores
    return rec_df

# Example usage (run and experiment):
# recs = recommend_by_name_item2vec("ocean eyes", candidate_index=2, top_k=15)
# display(recs)


In [ ]:
# Cell 9 — Prepare a sample of track embeddings for visualisation

# Take intersection of model vocab and metadata so we can colour by features
vocab_ids = set(model.wv.index_to_key)
meta_ids = set(track_meta["track_id"].unique())
vis_ids = list(vocab_ids & meta_ids)

print(f"Tracks usable for visualisation (in vocab & metadata): {len(vis_ids):,}")

# Sample subset to keep t-SNE manageable
N_TSNE = 3000
if len(vis_ids) > N_TSNE:
    vis_ids_sample = random.sample(vis_ids, N_TSNE)
else:
    vis_ids_sample = vis_ids

print(f"Using {len(vis_ids_sample)} tracks for t-SNE visualisation.")

# Build embedding matrix
emb_matrix = np.vstack([model.wv[tid] for tid in vis_ids_sample])

# Build a small metadata frame for the sampled tracks
vis_meta = (
    track_meta[track_meta["track_id"].isin(vis_ids_sample)]
    .drop_duplicates("track_id")
    .set_index("track_id")
    .loc[vis_ids_sample]  # preserve order
    .reset_index()
)

print("emb_matrix shape:", emb_matrix.shape)
display(vis_meta.head())


In [ ]:
# Cell 10 — Run t-SNE (this can take a bit of time)

tsne = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate=200,
    n_iter=1000,
    metric="euclidean",
    init="random",
    random_state=42,
)
X_tsne = tsne.fit_transform(emb_matrix)
print("X_tsne shape:", X_tsne.shape)

vis_meta["tsne_x"] = X_tsne[:, 0]
vis_meta["tsne_y"] = X_tsne[:, 1]


In [ ]:
# Cell 11 — Plot t-SNE, coloured by a feature (e.g. danceability)

# Merge in audio features from combined_features (if you want richer colouring).
# If you *don't* have combined_features handy in this notebook, skip the merge.
# Here I'll assume you do:

COMBINED_FEATURES_PATH = DATA_PROCESSED / "combined_features.csv"

cf = pd.read_csv(COMBINED_FEATURES_PATH)
cf_small = cf[["track_id", "danceability", "energy", "valence", "tempo"]].drop_duplicates("track_id")

vis = vis_meta.merge(cf_small, on="track_id", how="left")

fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(
    vis["tsne_x"],
    vis["tsne_y"],
    c=vis["danceability"],
    s=10,
    alpha=0.8,
)
plt.colorbar(sc, label="danceability")
ax.set_title("Item2Vec track embeddings (t-SNE projection) coloured by danceability")
ax.set_xlabel("t-SNE dim 1")
ax.set_ylabel("t-SNE dim 2")
plt.show()


In [ ]:
# Cell 12 — Another view: coloured by energy

fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(
    vis["tsne_x"],
    vis["tsne_y"],
    c=vis["energy"],
    s=10,
    alpha=0.8,
)
plt.colorbar(sc, label="energy")
ax.set_title("Item2Vec track embeddings (t-SNE projection) coloured by energy")
ax.set_xlabel("t-SNE dim 1")
ax.set_ylabel("t-SNE dim 2")
plt.show()


In [ ]:
# Cell 13 — Helper to plot a local neighbourhood for a given seed track

def plot_item2vec_neighbourhood(seed_track_id, top_k=15):
    """
    Plot seed track + its Item2Vec neighbours on the t-SNE scatter.
    """
    if seed_track_id not in model.wv:
        print("Seed track not in Item2Vec vocab.")
        return
    
    # nearest neighbours in embedding space
    sims = model.wv.most_similar(seed_track_id, topn=top_k)
    neighbour_ids = [tid for tid, _ in sims]
    neighbour_scores = [score for _, score in sims]
    
    # Ensure these ids exist in our vis DF
    ids_to_plot = [seed_track_id] + neighbour_ids
    mask = vis["track_id"].isin(ids_to_plot)
    sub = vis[mask].copy()
    
    # Label seed vs neighbours
    sub["type"] = np.where(sub["track_id"] == seed_track_id, "seed", "neighbour")
    
    fig, ax = plt.subplots(figsize=(7, 6))
    
    # Plot all background points faintly
    ax.scatter(vis["tsne_x"], vis["tsne_y"], s=5, alpha=0.1, label="others")
    
    # Plot neighbours
    nb = sub[sub["type"] == "neighbour"]
    ax.scatter(nb["tsne_x"], nb["tsne_y"], s=40, alpha=0.8, label="neighbours")
    
    # Plot seed
    sd = sub[sub["type"] == "seed"]
    ax.scatter(sd["tsne_x"], sd["tsne_y"], s=120, marker="*", label="seed", edgecolor="black")
    
    for _, row in sub.iterrows():
        ax.text(
            row["tsne_x"],
            row["tsne_y"],
            row["track_name"][:15],
            fontsize=7,
            alpha=0.9,
        )
    
    ax.set_title("Local Item2Vec neighbourhood (t-SNE space)")
    ax.legend()
    plt.show()

# Example:
# 1) Find a track-id via name
# candidates = find_track_ids_by_name("ocean eyes")
# display(candidates)

# 2) Pick the one you want:
# seed_tid = candidates.iloc[2]["track_id"]
# plot_item2vec_neighbourhood(seed_tid, top_k=15)


In [ ]:
# Cell 14 — Rough evaluation: can Item2Vec recover held-out songs?

RANDOM_STATE = 42
rng = np.random.default_rng(RANDOM_STATE)

pids = inter["pid"].unique()
SAMPLE_PLAYLISTS = 500
TOP_K = 20

sample_pids = rng.choice(pids, size=min(SAMPLE_PLAYLISTS, len(pids)), replace=False)

hits = 0
total = 0

for pid in sample_pids:
    seq = playlist_groups.loc[pid]
    if len(seq) < 6:
        continue

    # Very simple: treat last track as "held out"
    held_out = seq[-1]
    context_tracks = [tid for tid in seq[:-1] if tid in model.wv]

    if not context_tracks or held_out not in model.wv:
        continue

    # Combine neighbours from some context tracks
    candidate_scores = {}

    for tid in context_tracks[-5:]:  # last few tracks in playlist
        try:
            sims = model.wv.most_similar(tid, topn=TOP_K)
        except KeyError:
            continue
        for nb_tid, score in sims:
            candidate_scores[nb_tid] = max(candidate_scores.get(nb_tid, 0.0), score)

    # Top-K candidates overall
    if not candidate_scores:
        continue

    ranked = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    top_ids = [tid for tid, _ in ranked[:TOP_K]]

    total += 1
    if held_out in top_ids:
        hits += 1

if total > 0:
    hit_rate = hits / total
    print(f"Evaluated on {total} playlists.")
    print(f"HitRate@{TOP_K}: {hit_rate:.4f}")
else:
    print("Not enough playlists passed the filtering for evaluation.")


In [ ]:
recs = recommend_by_name_item2vec("hotel room service", candidate_index=1, top_k=15)
display(recs)

recs = recommend_by_name_item2vec("ocean eyes", candidate_index=2, top_k=15)
display(recs)
